# Baseline Learning - Basic Logistic Regresison Models

## Open Pickled Data

This is from what we extracted and pickled in Milestone 1 (updated to our [data extraction Notebook](https://colab.research.google.com/drive/1BInmWkNqitlIHXVlWWTid3fwA4y96J5j?usp=sharing)).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import pandas as pd

shared_folder_path = f'/content/drive/MyDrive/CS547 Project Group 11/pickled_data'
save_path = f'{shared_folder_path}/debugging_dataset_2023.pkl'

debugging_df = pd.read_pickle(save_path)
print(debugging_df.shape)

(1755354, 15)


## Model with input features: start_region

In [ ]:
# Preprocess dataframe for regions

NYC_REGION_TO_INT_MAP = {
    "Lower Manhattan": 0,
    "Midtown": 1,
    "Upper East Side": 2,
    "Upper West Side": 3,
    "East Village": 4,
    "West Village": 5,
    "Chelsea": 6,
    "Harlem": 7,
    "Downtown Brooklyn": 8,
    "North Brooklyn": 9
}

debugging_df.replace(NYC_REGION_TO_INT_MAP, inplace=True)

# INT_TO_NYC_REGION_MAP = {v: k for k, v in NYC_REGION_TO_INT_MAP.items()}

<ipython-input-6-e594c0c821d3>:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  debugging_df.replace(NYC_REGION_TO_INT_MAP, inplace=True)


In [ ]:
debugging_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_region,end_region
0,4F273B3B7376CE3A,classic_bike,2023-10-08 18:04:05.124,2023-10-08 18:07:46.599,91 St & 23 Ave,6949.04,24 Ave & 87 St,6885.01,40.767690,-73.879100,40.765400,-73.882560,member,2,2
1,91BFF1BDF63A207F,electric_bike,2023-10-16 15:21:28.991,2023-10-16 15:44:23.610,W 116 St & Broadway,7713.11,W 218 St & Broadway,8715.06,40.808200,-73.964100,40.871390,-73.914318,casual,7,7
2,CD8D2FD727DA0A80,electric_bike,2023-10-01 18:41:16.237,2023-10-01 18:54:34.150,W 26 St & 10 Ave,6382.05,W 45 St & 8 Ave,6676.02,40.749864,-74.003073,40.759291,-73.988597,member,6,1
3,3AA818DBFDCE15DE,electric_bike,2023-10-18 16:03:14.529,2023-10-18 16:23:40.034,35 Ave & 37 St,6563.12,W 43 St & 6 Ave,6593.14,40.755764,-73.923466,40.755273,-73.983169,casual,2,1
4,365D717CF2974497,electric_bike,2023-10-19 13:36:13.207,2023-10-19 14:00:39.103,Roosevelt Island Tramway,6599.01,3 Ave & E 62 St,6762.04,40.757229,-73.953508,40.763126,-73.965269,casual,2,2


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Train basic model to predict end_region based on start_region
X = debugging_df[['start_region']]
y = debugging_df['end_region']

# Split up Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, solver='saga'))
pipe.fit(X_train, y_train)

# Calculate base model accuracy
accuracy = pipe.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.2678688926171629


## Model with input features: start_region, start time (time of day, day of year)

In [ ]:
# Preprocessing code made with aid of ChatGPT
debugging_df['start_time_of_day'] = debugging_df['started_at'].dt.hour * 3600 + debugging_df['started_at'].dt.minute * 60 + debugging_df['started_at'].dt.second
debugging_df['start_day_of_year'] = debugging_df['started_at'].dt.dayofyear

debugging_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_region,end_region,start_time_of_day,start_day_of_year
0,4F273B3B7376CE3A,classic_bike,2023-10-08 18:04:05.124,2023-10-08 18:07:46.599,91 St & 23 Ave,6949.04,24 Ave & 87 St,6885.01,40.767690,-73.879100,40.765400,-73.882560,member,2,2,65045,281
1,91BFF1BDF63A207F,electric_bike,2023-10-16 15:21:28.991,2023-10-16 15:44:23.610,W 116 St & Broadway,7713.11,W 218 St & Broadway,8715.06,40.808200,-73.964100,40.871390,-73.914318,casual,7,7,55288,289
2,CD8D2FD727DA0A80,electric_bike,2023-10-01 18:41:16.237,2023-10-01 18:54:34.150,W 26 St & 10 Ave,6382.05,W 45 St & 8 Ave,6676.02,40.749864,-74.003073,40.759291,-73.988597,member,6,1,67276,274
3,3AA818DBFDCE15DE,electric_bike,2023-10-18 16:03:14.529,2023-10-18 16:23:40.034,35 Ave & 37 St,6563.12,W 43 St & 6 Ave,6593.14,40.755764,-73.923466,40.755273,-73.983169,casual,2,1,57794,291
4,365D717CF2974497,electric_bike,2023-10-19 13:36:13.207,2023-10-19 14:00:39.103,Roosevelt Island Tramway,6599.01,3 Ave & E 62 St,6762.04,40.757229,-73.953508,40.763126,-73.965269,casual,2,2,48973,292


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Train basic model to predict end_region using multiple features
X = debugging_df[['start_region', 'start_time_of_day', 'start_day_of_year']]
y = debugging_df['end_region']

# Split up Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, solver='saga'))
pipe.fit(X_train, y_train)

# Calculate base model accuracy
accuracy = pipe.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.2907702430562479
